## About this test
This test shall take as input a directory containing neuronal morphologies (currently supports `swc` format). The user decides whether to run the validations for all available morphologies, or a subset of these.

The validations are carried out using 'NeuroM' (https://github.com/BlueBrain/NeuroM). The validation test evaluates the morphology in two stages:

1. **Hard Constraints**

   Here we evaluate the integrity of the neuronal reconstruction in order to determine if it is appropriate for further evaluations. The evaluations here can be sub-divided into the following NeuroM features (apps):
   
   - morph_check
   - cut_plane_detection <br /><br />
   
2. **Soft Constraints** [Currently only available for *Fast Spiking Interneurons*]

     Neuronal reconstructions that pass the 'Hard Constraints' are evaluated here for their morphometric features. The features are extracted using NeuroM's morph_stats app, wherever possible, either directly or as a combination of multiple features. These are then compared against experimentally obtained data, as determined by the particular validation test being executed.
     
     Some of the features currently included are soma's diameter and the maximal branch order in the dendrites, besides the number of trunk sections, -X,Y,Z- extents, field's diameter and total path-length of both the axon and the dendrites.
     
     By the present, only FS interneurons can be considered, since observation data is missing for other neuron types.

### Installing the required packages
(no user intervention required)

In [ ]:
import os
import shutil

import pkg_resources
from pkg_resources import parse_version

req_packages = {    
                    "hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"},
                    "sciunit"                  : {"min_version": "0.19",   "install_version": "https://github.com/scidash/sciunit"},
                    "morphounit"               : {"min_version": "1.0",    "install_version": "1.0"},
                    "hbp_validation_framework" : {"min_version": "0.4",    "install_version": "0.4.1"},
                    "fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"},
                    "PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"}
               }

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg_name, pkg_vinfo in req_packages.items():
        print("Checking for package: {}".format(pkg_name))
        try:
            pkg_resources.get_distribution(pkg_name)
            if pkg_vinfo["min_version"]:
                current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
                if current_version < parse_version(pkg_vinfo["min_version"]):
                    print("\t{}: current version = {}".format(pkg_name, current_version))
                    print("\tA newer version of {} will be installed.".format(pkg_name))
                    raise
        except:
            if "github.com" in pkg_vinfo["install_version"]:
                os.system("pip install --quiet git+{}".format(pkg_vinfo["install_version"]))                
            else:
                os.system("pip install --quiet {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                
            if pkg_name == "hbp_service_client":
                from IPython.display import HTML
                display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))
                
install_req_packages()                 

import json
import sciunit
from morphounit.utils import neuroM_loader, NeuroM_MorphStats
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

### Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME, environment="production")
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_collab_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_collab_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

### Model Selection
The user is given an option to choose from existing models, or to specify a new model. The former are currently sourced from https://collab.humanbrainproject.eu/#/collab/8123/nav/61645 (see: `/8123/usecases/models/basal_morphology/`). Only models saved at this location are shown as existing models.

If specifying a new model, the path to the model directory within Collab storage needs to be specified.<br />
E.g. `/8123/usecases/models/basal_morphology/msn_d1/`

In [ ]:
# print("Choose from the following options:")
options = ["D1-type Medium Spiny Neurons", "D2-type Medium Spiny Neurons", "Fast Spiking Interneurons"]
options.append("Use own model directory (Collab path to be specified)")
for i, each in enumerate(options,start=1):
    print ("\t{}. {}".format(i,each))
print("Enter the # of required option: ")
choice = input()

if choice == 1:
    print ("\nInput: D1-type Medium Spiny Neurons")
    morph_dir = "/8123/usecases/models/basal_morphology/msn_d1/"
elif choice == 2:
    print ("\nInput: D2-type Medium Spiny Neurons")
    morph_dir = "/8123/usecases/models/basal_morphology/msn_d2/"        
elif choice == 3:
    print ("\nInput: Fast Spiking Interneurons")
    morph_dir = "/8123/usecases/models/basal_morphology/fs/"            
else:
    print("Please enter the Collab storage path to the directory containing morphologies:")
    print("e.g. '/8123/usecases/models/basal_morphology/msn_d1/'")
    morph_dir = raw_input()
    
print("Choose from the following available morphologies:")
clients = get_hbp_service_client()
file_entries = clients.storage.list(morph_dir)
file_entries = [f for f in file_entries if f.endswith(".swc")]
halfLen = int(len(file_entries)/2)
for i, each in enumerate(zip(file_entries,file_entries[halfLen:]),start=1):
    print ("{0:>3}. {2:>40} \t {1:>3}. {3:>40}".format(i,i+halfLen,*each))
print("\nEnter the # of required option: ")
print("Example inputs: 1, 4, [1,4,8,9], all")
file_choice = raw_input()

if file_choice == "all":
    morph_inds = range(len(file_entries))
else:    
    if isinstance(eval(file_choice), list):
        morph_inds = eval(file_choice)
        morph_inds = [i-1 for i in morph_inds]
    elif isinstance(eval(file_choice), int):
        morph_inds = [eval(file_choice)-1]
    else:
        raise ValueError("Invalid entry for morphology choice!")        

morph_files = []
for ind in morph_inds:
    morph_entry = os.path.basename(file_entries[ind])
    file_path = "/tmp/" + morph_entry
    clients.storage.download_file(morph_dir+morph_entry, file_path)   
    print "morph_dir+morph_entry = ", morph_dir+morph_entry
    print "file_path = ", file_path
    morph_files.append(file_path)

### Gather Additional Info
Currently the test is applicable for three cell types: _D1-type MSN_, _D2-type MSN_ and _FS Interneurons_ <br />
The usecase tries to identify the cell type from the input directory. If this isn't possible, the user is prompted to specify the cell type.

In [ ]:
# will try to determine cell type from file path; if not then ask user
# current options: msn_d1, msn_d2, fs
cell_type_map = {"msn_d1" : "Medium spiny neuron (D1 type)",
                 "msn_d2" : "Medium spiny neuron (D2 type)",
                 "fs"     : "Fast spiking interneuron"}

if "/msn_d1" in morph_dir:
    cell_type = "msn_d1"     
elif "/msn_d2" in morph_dir:
    cell_type = "msn_d2"
elif "/fs" in morph_dir:
    cell_type = "fs"    
else:
    print("\nPlease enter the cell_type: ")
    options = ["msn_d1", "msn_d2", "fs"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = input()
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))

### Specifying model from ModelCatalog; Update info if new model entry
For a model to use the validation framework, it first has to be registered on the _Model Catalog_ app (see Collab's Navigation panel; select `Model Catalog`) <br />

If the model has already been registered on the model catalog, then you simply need to specify the `model_uuid`. <br /> E.g. c7748201-5551-4d56-bc31-f7feec5e5798 <br />

If this is to be registered as a new model, then you are required to update the cell below to alter the model metadata as required. See [here](http://hbp-validation-client.readthedocs.io/en/master/#hbp_validation_framework.ModelCatalog.register_model) for detailed info on the various parameters. Once a new model is registered on the model catalog, it will show you the corresponding `model_uuid`. This can be used for future tests to link the results with the same model entry.

In [ ]:
print("\nDo you wish to use an existing `model_uuid` (this will register results against an existing model entry)?")
print("Note: `model_uuid` is obtained from the HBP Validation Framework app.")
print("If not specifying `model_uuid`, then a new model entry would be registered.")
print("Note: If registering a new model, please verify the default model metadata in this Jupyter notebook!")
print("Enter: y/n")
choice = raw_input().lower()
valid_choices = {"yes": True, "y": True, "no": False, "n": False}
if valid_choices[choice]:
    print("\nPlease enter a valid `model_uuid`: ")
    model_uuid = raw_input()
    # Verify the user specified `model_uuid` is valid
    try:
        modelCatalog.get_model(model_id=model_uuid)
    except:
        raise ValueError("Specified model_uuid = {} is invalid!".format(model_uuid))
else:
    # Create a new model entry in the ModelCatalog; metadata to be used for registering model
    model_uuid = modelCatalog.register_model(
                                app_id=str(MCapp_navID),
                                name=os.path.split(os.path.dirname(morph_dir))[-1]+"_morphologies",
                                # alias="", # optional; ensure uniqueness                                
                                author="Alexander Kozlov",
                                organization="KTH-UNIC",
                                private=False,
                                cell_type= cell_type_map.get(cell_type, "Other"),
                                model_type="Model Collection",
                                brain_region="Basal Ganglia",
                                species="Mouse (Mus musculus)",
                                description="This model is being used to demonstrate use of the Validation Service")
    model_uuid = str(model_uuid)
    print("\nModel registered on the HBP Validation Framework's Model Catalog!")
    print("Note: you can specify the below `model_uuid` for future runs of this test.")
print("\nmodel_uuid = {}".format(model_uuid))
print("Link to ModelCatalog app:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=model.{}".format(str(collab_id),str(MCapp_navID),model_uuid))

## Part 1: Hard Constraints Validation

### Instantiating the model; Running the validation tests
Validations for hard constraints are run for each of the selected morphologies. At the end of the test, the user is provided with a textual summary of the _score_ and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
result_uuids = []

# cycling through all selected morphologies
for morph_path in morph_files:
    model_name = os.path.basename(morph_path)[:-4]
    morph_model = neuroM_loader(model_path=file_path, name=model_name)
    morph_model.model_uuid = model_uuid
    morph_model.source = "http://collab.link" + morph_dir+os.path.basename(morph_path)    
    morph_model.version = model_name
    
    if cell_type in ["msn_d1", "msn_d2"]:
        test_alias = "basalg_msn_morph_hardChecks"
    else:
        test_alias = "basalg_fs_morph_hardChecks"
    result_id = utils.run_test(hbp_username=HBP_USERNAME, environment="production", model=morph_model, test_alias=test_alias, test_version="1.0", storage_collab_id=collab_id, register_result=True, client_obj=testLibrary)
    result_uuids.append(result_id)    

print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### Generate Report
The validation framework can generate a PDF report for all model variants (HoF parameter sets employed) that successfully completed the test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        if not HBP_USERNAME:
            print("Enter your HBP username: ")
            HBP_USERNAME = raw_input()  
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="production", result_list=result_uuids, only_combined=True)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))

## Part 2: Soft Constraints Validation
[Currently only available for Fast Spiking Interneurons]

In [ ]:
if cell_type != "fs":
    print("Soft constraints validation not currently available for cell type = {}".format(cell_type))
    result_uuids = []

### Instantiating the model; Running the validation tests
Validations for soft constraints are run for only the morphologies that _pass_ the first part of the test. At the end of the test, the user is provided with a textual summary of the _mean score_ value (across the features tested) and the path to related output files generated by the test. These and other details can be viewed in the  _Validation Framework_ app (see Collab's Navigation panel; select `Validation Framework`).

In [ ]:
# Obtain instance_uuid of all above morphologies that passed the hard constraints validations
# Only these will be evaluated for the soft constraints
passed_morph_inst_uuid = []
for uuid in result_uuids:
    result = testLibrary.get_result(result_id=uuid)
    if result["results"][0]["passed"]:
        passed_morph_inst_uuid.append(result["results"][0]["model_version_id"])

In [ ]:
model_collab_paths = []
for uuid in passed_morph_inst_uuid:
    model_instance = modelCatalog.get_model_instance(instance_id=uuid)
    model_collab_paths.append(model_instance["source"].split("http://collab.link")[1])

In [ ]:
models_path = '/tmp/neuroM_morph_softChecks'
if os.path.exists(models_path):
    shutil.rmtree(models_path)
os.makedirs(models_path)

# Downloading morphology files that passed the hard constraints validations
for morph_file in model_collab_paths:
    # Downloading a morphology file that passed the hard constraints validations
    morph_path = os.path.join(models_path, os.path.basename(morph_file))
    clients.storage.download_file(str(morph_file), morph_path)

# Downloading configuration file for 'morph_stats'
config_file = 'FSI_morph_stats_config.yaml'
test_obs_path = '/8123/test_observations/morphounit'
config_collab = os.path.join(test_obs_path, config_file)
config_path = os.path.join(models_path, config_file)
clients.storage.download_file(config_collab, config_path)

# Cycling through all above morphologies that passed the hard constraints validations
test_alias = 'morph_stats_Test'

morph_stats_file = 'NeuroM_MorphStats_output.json'

result_uuids = []

file_entries = os.listdir(models_path)
morph_files = [f for f in file_entries if f.endswith(".swc")]
for file_name in morph_files:
    
    morph_path = os.path.join(models_path, file_name)
    model_name = file_name.split(".")[0] # Cell-ID
    
   
    morph_model = NeuroM_MorphStats(model_name=model_name, morph_path=morph_path, 
                                     config_path=config_path, morph_stats_file=morph_stats_file)
    
    morph_model.model_uuid = model_uuid  # uuid for the whole cell collection
    morph_model.version = model_name     # basename of each morphology file
    
    result_id = utils.run_test(hbp_username=HBP_USERNAME, environment="production", 
                               model=morph_model, test_alias=test_alias, test_version="2.0",
                               storage_collab_id=collab_id, register_result=True, 
                               client_obj=testLibrary)
        
    result_uuids.append(result_id)
    
if len(result_uuids) > 0:   
    print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))    

### Generate Report
The validation framework can generate a PDF report for all model variants that successfully completed the test. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        if not HBP_USERNAME:
            print("Enter your HBP username: ")
            HBP_USERNAME = raw_input()  
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="production", 
                                                         result_list=result_uuids, only_combined=True)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
if report_path:
    print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))